In [6]:
import pandas as pd
import numpy as np
import torch
import csv
from util_v3 import *
from data_v3 import load_data, get_train_datasets
from model_v3 import *
from torch.utils.data import dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable

# from mscn.data import load_data

import torch
import torch.nn as nn
import torch.nn.functional as F
# from mscn.data import get_train_datasets, load_data, make_dataset

## load and encode train data by calling the function

In [7]:
dataset_name = 'tpcds'

dicts, column_min_max_vals, labels_train, labels_test, max_num_joins, max_num_predicates, train_data, test_data = get_train_datasets(dataset_name)

Loaded queries
Number of training samples: 9336
Number of validation samples: 1038
Created TensorDataset for training data
Created TensorDataset for validation data


In [8]:
num_epochs = 10
batch_size = 1
hid_units = 256
cuda = False

num_materialized_samples = 0

num_classes = 19

In [9]:
table2vec, column2vec, op2vec, join2vec = dicts

num_materialized_samples = 0

# Train model
sample_feats = len(table2vec) + num_materialized_samples
predicate_feats = len(column2vec) + len(op2vec) + 1
join_feats = len(join2vec)

model = SetConv(sample_feats, predicate_feats, join_feats, hid_units, num_classes)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
# test_data_loader = DataLoader(test_data, batch_size=batch_size)

model.train()

SetConv(
  (sample_mlp1): Linear(in_features=26, out_features=256, bias=True)
  (sample_mlp2): Linear(in_features=256, out_features=256, bias=True)
  (predicate_mlp1): Linear(in_features=30, out_features=256, bias=True)
  (predicate_mlp2): Linear(in_features=256, out_features=256, bias=True)
  (join_mlp1): Linear(in_features=56, out_features=256, bias=True)
  (join_mlp2): Linear(in_features=256, out_features=256, bias=True)
  (out_mlp1): Linear(in_features=768, out_features=256, bias=True)
  (out_mlp2): Linear(in_features=256, out_features=19, bias=True)
)

In [10]:
loss_fn = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    accuracy_hist_train = 0

    for batch_idx, data_batch in enumerate(train_data_loader):

        samples, predicates, joins, targets, sample_masks, predicate_masks, join_masks = data_batch

        samples, predicates, joins, targets = Variable(samples), Variable(predicates), Variable(joins), Variable(targets)            
        sample_masks, predicate_masks, join_masks = Variable(sample_masks), Variable(predicate_masks), Variable(join_masks)

        outputs = model(samples, predicates, joins, sample_masks, predicate_masks, join_masks)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        is_correct = (torch.argmax(outputs, dim=1) == targets).float()
        accuracy_hist_train += is_correct.sum()
    accuracy_hist_train /= len(train_data_loader)

    print(f'Epoch {epoch} Accuracy ' f'{accuracy_hist_train: .4f}')

Epoch 0 Accuracy  0.5660
Epoch 1 Accuracy  0.6112
Epoch 2 Accuracy  0.5799
Epoch 3 Accuracy  0.5799
Epoch 4 Accuracy  0.5799
Epoch 5 Accuracy  0.5799
Epoch 6 Accuracy  0.5799
Epoch 7 Accuracy  0.5799
Epoch 8 Accuracy  0.5799
Epoch 9 Accuracy  0.5614
